In [1]:
import requests
import pandas as pd
import numpy as np
import json
from pathlib import Path
import keepa
import datetime
import matplotlib.pyplot as plt
import math
import os
import keepa

In [13]:
# function remove extra subcategories from each row and rename columns
def clean_frame(df):
    df = df.rename(columns = {'Title' : 'product_title', 'Categories: Sub' : 'subcategory', 'ASIN' : 'asin'})
    def clean_row(row):
        row['subcategory'] = row['subcategory'].split(',')[0].strip().lower()
        return row
    df = df.apply(clean_row, axis = 1)
    
    return df

In [14]:
def keepa_time_to_datetime(kt):
    # Convert Keepa time (minutes since 2011-01-01) to a Python datetime (UTC)
    if isinstance(kt, datetime.datetime):
        return kt
    return datetime.datetime.fromtimestamp((kt + 21564000) * 60, datetime.timezone.utc)

def generate_monthly_headers(days):
    """
    Generate a list of month headers (strings) in the format 'YYYY-MM'
    spanning from the current month back to the month that includes (now - days).
    The headers are in ascending order. 2021-2025
    This function standardizes which months we collect for each batch and ensures the columns are aligned.
    """
    now = datetime.datetime.now(datetime.timezone.utc)
    start_date = now - datetime.timedelta(days=days)
    
    headers = []
    current_year = now.year
    current_month = now.month

    while True:
        header = f"{current_year:04d}-{current_month:02d}"
        headers.append(header)
        # Move to the previous month
        if current_month == 1:
            current_month = 12
            current_year -= 1
        else:
            current_month -= 1
        
        # Create a timezone-aware date for the first day of the new month.
        month_start = datetime.datetime(current_year, current_month, 1, tzinfo=datetime.timezone.utc)
        # Stop if this month is before the start_date.
        if month_start < start_date:
            break
    return sorted(headers)


def get_monthly_avg_prices(asins, days=1460):
    """
    asins: list of ASIN strings
    days: number of days of history (default 1460 ~ 4 years)
    
    Returns a DataFrame:
        - Rows = ASINs
        - Columns = monthly time periods (e.g. '2025-02', '2025-01', etc.)
        - Values = average 'NEW' price for that month
    """
    products = api.query(asins, days=days)
    dfs = []
    for product in products:
        asin = product['asin']
        price_history = product['data'].get('NEW', [])
        time_history  = product['data'].get('NEW_time', [])
        
        dates = [keepa_time_to_datetime(t) for t in time_history]
        prices = [p for p in price_history]
        
        df = pd.DataFrame({'date': dates, asin: prices})
        df.set_index('date', inplace=True)
        
        # Resample to monthly average using month-end frequency
        monthly_avg = df.resample('ME').mean()
        dfs.append(monthly_avg)
    
    if not dfs:
        return pd.DataFrame()
    
    # Combine and transpose so that rows = ASIN and columns = dates
    combined = pd.concat(dfs, axis=1).T
    # Convert datetime columns to string format 'YYYY-MM'
    combined.columns = [col.strftime('%Y-%m') for col in combined.columns]
    
    # Generate the complete set of monthly headers (headers are in descending order)
    headers = generate_monthly_headers(days)
    
    # Reindex with headers generated headers
    combined = combined.reindex(columns=headers, fill_value=np.nan)
    
    # Forward fill missing values along the row (in chronological order)
    combined = combined.ffill(axis=1)
    
    return combined


def batch(iterable, n=20):
    """Yield successive n-sized chunks from iterable."""
    for i in range(0, len(iterable), n):
        yield iterable[i:i + n]

In [15]:
def query_keepa_in_batches(products, category, max_batches=10, batch_size=20,
                           days=365 * 4, start_index=0, stop_index=None):
    """
    Query Keepa for monthly average prices in batches and incrementally save 
    the *merged* results (column-aligned) to a CSV file.
    """
    # Slice the ASIN list based on start_index and stop_index.
    asins = list(products['asin'])[start_index:stop_index]
    csv_file = f'data/asin_prices/{category}_monthly_prices.csv'
    for i, asin_batch in enumerate(batch(asins, batch_size)):
        if i >= max_batches:
            break
        df_batch = get_monthly_avg_prices(asin_batch, days=days)
        if df_batch.empty:
            print(f"Batch {i+1} returned no data; skipping.")
            continue
        # If the CSV file exists, read it, merge columns, then write back
        if os.path.exists(csv_file):
            existing_df = pd.read_csv(csv_file, index_col=0)
            
            # Merge on row index (ASIN) and combine columns (outer join).
            # combine_first() will fill missing entries in existing_df with df_batch values.
            merged_df = existing_df.combine_first(df_batch)
            
            # Ensure columns are in the correct order (descending monthly headers).
            # This step uses the same monthly headers function to reindex.
            headers = generate_monthly_headers(days)
            merged_df = merged_df.reindex(columns=headers, fill_value=np.nan)
            
            merged_df.to_csv(csv_file, index=True)
        else:
            # If no CSV yet, just write df_batch as the first chunk
            df_batch.to_csv(csv_file, index=True)
        
        print(f"Batch {i+1} processed and merged.")


# Example helper function to split the ASIN list into batches.
def batch(iterable, n=20): # 20 is default batch size
    """Yield successive n-sized chunks from iterable"""
    for i in range(0, len(iterable), n):
        yield iterable[i:i + n]

In [16]:
def clean_prices(df):
    # Rename 'Unnamed: 0' to 'asin'
    df = df.rename(columns={'Unnamed: 0': 'asin'})
    
    # Round every value in all columns except the first (asin) to 2 decimal places
    df.iloc[:, 1:] = df.iloc[:, 1:].round(2)
    
    return df

In [43]:
ACCESS_KEY = "df2mtauj1tmrngcm95ubshd41fplpf2bfh1nba8s8hpd2m6golbbrj9bat7osb8o" # do no share outside of private repo!!
api = keepa.Keepa(ACCESS_KEY, timeout=30)

In [44]:
category = 'home_and_kitchen'
data_path = Path('data/keepa_data') / f'{category}.csv'
products = pd.read_csv(data_path)
products = clean_frame(products)

In [48]:
query_keepa_in_batches(products, category, max_batches=4, batch_size=20, days = (365 * 5) + 60, start_index = 420, stop_index = None)



100%|██████████| 20/20 [00:05<00:00,  3.34it/s]


Batch 1 processed and merged.




100%|██████████| 20/20 [00:04<00:00,  4.19it/s]


Batch 2 processed and merged.




100%|██████████| 20/20 [00:04<00:00,  4.12it/s]


Batch 3 processed and merged.



Waiting 475 seconds for additional tokens

100%|██████████| 20/20 [08:03<00:00, 24.17s/it]

Batch 4 processed and merged.


In [ ]:
# categories = ['appliances','electronics', 'grocery_and_foods', 'pet_supplies', 'toys', 'beauty_and_personal_care', 'clothing_shoes_and_jewelry','baby_products', 'home_and_kitchen']

In [135]:
category = 'home_and_kitchen'

In [136]:
data_path2 = Path('data/asin_prices') / f'{category}_monthly_prices.csv'
price_data = pd.read_csv(data_path2)
price_data = clean_prices(price_data)
price_data

,asin,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,...,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03
0,B00004OCIP,9.73,9.97,15.46,13.32,13.32,13.32,13.32,16.27,9.82,...,9.98,10.57,11.16,10.30,9.69,9.75,9.93,10.28,10.39,11.33
1,B00005UP2M,335.10,355.48,331.64,377.02,328.72,409.00,384.48,359.95,359.95,...,349.97,351.27,379.95,381.54,351.65,331.08,361.37,449.97,449.97,449.97
2,B0000645YL,32.99,31.35,39.04,39.94,38.94,37.29,37.29,37.29,39.87,...,35.99,33.92,33.92,38.95,33.65,35.98,39.27,32.99,39.29,39.29
3,B00006JKZN,31.60,26.95,45.93,41.21,40.55,36.99,38.18,37.47,37.47,...,28.82,28.23,29.70,29.22,27.67,29.42,30.24,28.45,25.97,27.55
4,B00006JSUB,23.46,23.44,24.86,24.22,24.22,23.17,24.80,24.80,24.80,...,26.12,26.17,25.48,26.52,24.70,25.81,27.50,27.58,28.33,28.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,B08MW9ZVV3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13.99,13.99,13.99,13.99,13.99,13.99,13.99,13.99,13.99,13.99
496,B08N9Q24M9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,29.54,29.98,29.05,34.26,29.98,42.10,27.42,24.86,27.41,29.97
497,B08NBBSNTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,67.99,67.99,56.16,49.99,49.99,49.99,49.99,49.99,49.99,49.99
498,B08NWGNQPS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,52.58,43.25,46.48,48.95,48.98,43.98,48.95,42.65,38.59,38.59


In [137]:
cols_to_drop = list(price_data.columns[1:12]) + list(price_data.columns[-3:])
price_data = price_data.drop(columns=cols_to_drop)

In [138]:
price_data

,asin,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,B00004OCIP,9.78,9.76,9.92,9.98,9.95,9.65,9.90,10.79,10.85,...,11.36,11.25,11.29,9.98,10.57,11.16,10.30,9.69,9.75,9.93
1,B00005UP2M,359.95,359.95,359.95,359.95,359.95,359.95,359.95,448.20,583.70,...,381.06,362.54,336.28,349.97,351.27,379.95,381.54,351.65,331.08,361.37
2,B0000645YL,37.17,37.17,37.17,38.99,39.88,39.28,39.95,39.95,37.14,...,39.92,39.92,39.92,35.99,33.92,33.92,38.95,33.65,35.98,39.27
3,B00006JKZN,33.33,29.93,37.65,29.92,29.40,29.81,29.75,29.78,23.19,...,27.74,29.32,28.85,28.82,28.23,29.70,29.22,27.67,29.42,30.24
4,B00006JSUB,25.50,26.25,26.82,24.23,30.02,28.65,27.54,27.54,28.81,...,32.40,32.32,28.72,26.12,26.17,25.48,26.52,24.70,25.81,27.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,B08MW9ZVV3,13.99,13.99,13.99,14.39,14.59,14.59,14.59,14.59,14.59,...,13.99,13.99,13.99,13.99,13.99,13.99,13.99,13.99,13.99,13.99
496,B08N9Q24M9,29.98,24.69,25.97,24.68,25.11,26.47,28.97,28.97,29.94,...,29.70,28.90,28.84,29.54,29.98,29.05,34.26,29.98,42.10,27.42
497,B08NBBSNTC,74.99,74.99,72.99,72.99,75.99,75.99,76.99,76.99,76.99,...,67.99,67.99,67.99,67.99,67.99,56.16,49.99,49.99,49.99,49.99
498,B08NWGNQPS,49.00,39.00,39.00,44.00,40.67,39.25,39.25,54.78,68.76,...,42.28,45.48,48.95,52.58,43.25,46.48,48.95,48.98,43.98,48.95


In [139]:
data_path = Path('data/keepa_data') / f'{category}.csv'
products = pd.read_csv(data_path)
products = clean_frame(products)

In [140]:
products

,product_title,subcategory,asin
0,"Amazon Basics Bed Sheets Queen Size, Deluxe Mi...",sheet & pillowcase sets,B07QT9PKC6
1,Owala FreeSip Insulated Stainless Steel Water ...,thermoses,B085DTZQNZ
2,"AmazonBasics Microfiber Sheet Set, Twin, Brigh...",sheet & pillowcase sets,B0154AS4T4
3,Twin Size 3 Piece Sheet Set - Comfy Breathable...,sheet & pillowcase sets,B07BGSZPHZ
4,Etekcity Smart Food Kitchen Scale with Nutriti...,digital scales,B07FCZSC41
...,...,...,...
995,"Bernhard Products Black Wall Clock, Large 13-I...",wall clocks,B07NPXRGBH
996,Bedsure Fluffy Green King Size Blanket for Bed...,throws,B08CY7WZ43
997,Dupray Neat Steam Cleaner with 17-Piece Kit – ...,steam cleaners,B07C44DM6D
998,"Frosted Clear Table Protector 24 x 40 Inch, 1....",table pads,B07NZ2ZHB9


In [141]:
merged_data = pd.merge(products, price_data, on='asin', how='left')

In [142]:
merged_data = merged_data.dropna(subset=merged_data.columns[4:], how='all').reset_index(drop=True)

In [143]:
merged_data

,product_title,subcategory,asin,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,"Amazon Basics Bed Sheets Queen Size, Deluxe Mi...",sheet & pillowcase sets,B07QT9PKC6,18.99,19.99,19.99,19.99,26.79,20.71,20.71,...,17.73,16.60,16.49,15.99,16.50,16.50,16.50,16.50,15.30,16.50
1,Owala FreeSip Insulated Stainless Steel Water ...,thermoses,B085DTZQNZ,23.97,23.97,20.30,21.99,26.74,22.37,23.97,...,19.81,35.97,34.14,33.03,27.83,27.58,27.11,23.17,25.35,27.15
2,"AmazonBasics Microfiber Sheet Set, Twin, Brigh...",sheet & pillowcase sets,B0154AS4T4,14.99,14.99,14.99,14.99,14.99,14.99,14.99,...,14.00,13.24,13.24,13.99,12.99,12.99,12.99,12.99,10.61,12.99
3,Twin Size 3 Piece Sheet Set - Comfy Breathable...,sheet & pillowcase sets,B07BGSZPHZ,25.19,25.19,25.19,25.89,25.89,25.89,24.10,...,25.18,25.18,25.89,25.89,25.89,25.89,25.89,25.89,23.79,27.26
4,Etekcity Smart Food Kitchen Scale with Nutriti...,digital scales,B07FCZSC41,29.99,25.97,29.94,26.87,25.98,29.99,29.99,...,39.99,39.99,42.02,39.99,39.98,39.98,39.98,39.85,37.82,38.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,JOLLYVOGUE King Size Pillows for Sleeping Set ...,bed pillows,B07TLVGQQ7,36.35,36.60,36.17,36.17,36.99,36.99,36.99,...,30.49,29.74,29.32,28.32,31.49,27.48,27.48,29.05,32.99,29.99
495,"Hearth & Harbor Queen Sheets Sets - 6 Piece, E...",sheet & pillowcase sets,B07SYVP2MX,56.54,28.77,28.77,28.77,28.77,28.77,34.70,...,30.48,23.31,24.68,26.31,29.94,25.79,27.69,27.01,25.39,23.58
496,Handmade with Love by Fatima. As seen in Monic...,picture frames,B07Z47DKJT,26.95,26.95,26.95,26.95,26.95,26.95,26.95,...,26.95,26.95,26.95,26.95,26.95,26.95,26.95,26.95,26.95,26.95
497,Kitchen Gizmo Snap N Strain Pot Strainer and P...,food strainers,B08F6Z32DL,16.49,16.49,16.49,15.12,15.39,15.39,14.08,...,14.78,14.70,9.99,9.24,10.66,12.66,15.49,12.12,18.07,16.99


In [144]:
folder_path = "data/cleaned_keepa_data"
file_path = os.path.join(folder_path, f"{category}_cleaned.csv")
merged_data.to_csv(file_path, index=False)  

print(f"Data saved to {file_path}")

Data saved to data/cleaned_keepa_data/home_and_kitchen_cleaned.csv
